In [1]:
!pip install xgboost

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [3]:
players=pd.read_csv(r"players.csv")
matches=pd.read_csv(r"match.csv")
players.head()

,match_id,account_id,hero_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,kills,deaths,...,unit_order_glyph,unit_order_eject_item_from_stash,unit_order_cast_rune,unit_order_ping_ability,unit_order_move_to_direction,unit_order_patrol,unit_order_vector_target_position,unit_order_radar,unit_order_set_item_combine_lock,unit_order_continue
0,0,0,86,0,3261,10960,347,362,9,3,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,51,1,2954,17760,494,659,13,3,...,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,83,2,110,12195,350,385,0,4,...,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2,11,3,1179,22505,599,605,8,4,...,1.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,3,67,4,3307,23825,613,762,20,3,...,3.0,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
players=players[["match_id","hero_id","player_slot"]]
players

,match_id,hero_id,player_slot
0,0,86,0
1,0,51,1
2,0,83,2
3,0,11,3
4,0,67,4
...,...,...,...
499995,49999,100,128
499996,49999,9,129
499997,49999,90,130
499998,49999,73,131


In [5]:
players.isnull().values.any()

False

In [6]:
players.dtypes

match_id       int64
hero_id        int64
player_slot    int64
dtype: object

In [7]:
def get_position(x):
    if x>=128:
        x-=128
    return x
players["position"]=players["player_slot"].apply(lambda x: get_position(x))
players["radiant"]=players["player_slot"].apply(lambda x: x<128)
players.head(10)

,match_id,hero_id,player_slot,position,radiant
0,0,86,0,0,True
1,0,51,1,1,True
2,0,83,2,2,True
3,0,11,3,3,True
4,0,67,4,4,True
5,0,106,128,0,False
6,0,102,129,1,False
7,0,46,130,2,False
8,0,7,131,3,False
9,0,73,132,4,False


In [8]:
matches.head()

,match_id,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win,negative_votes,positive_votes,cluster
0,0,1446750112,2375,1982,4,3,63,1,22,True,0,1,155
1,1,1446753078,2582,0,1846,63,0,221,22,False,0,2,154
2,2,1446764586,2716,256,1972,63,48,190,22,False,0,0,132
3,3,1446765723,3085,4,1924,51,3,40,22,False,0,0,191
4,4,1446796385,1887,2047,0,0,63,58,22,True,0,0,156


In [9]:
matches=matches[["match_id","radiant_win"]]

In [10]:
matches.head()

,match_id,radiant_win
0,0,True
1,1,False
2,2,False
3,3,False
4,4,True


In [11]:
data=pd.merge(players,matches,left_on="match_id",right_on="match_id",how="left")
data.head()

,match_id,hero_id,player_slot,position,radiant,radiant_win
0,0,86,0,0,True,True
1,0,51,1,1,True,True
2,0,83,2,2,True,True
3,0,11,3,3,True,True
4,0,67,4,4,True,True


In [12]:
data=data[["match_id","hero_id","position","radiant","radiant_win"]]

In [13]:
data.head()

,match_id,hero_id,position,radiant,radiant_win
0,0,86,0,True,True
1,0,51,1,True,True
2,0,83,2,True,True
3,0,11,3,True,True
4,0,67,4,True,True


In [14]:
one_row=data.groupby('match_id')['hero_id'].agg(list).reset_index()
one_row.head()

,match_id,hero_id
0,0,"[86, 51, 83, 11, 67, 106, 102, 46, 7, 73]"
1,1,"[7, 82, 71, 39, 21, 73, 22, 5, 67, 106]"
2,2,"[51, 109, 9, 41, 27, 38, 7, 10, 12, 85]"
3,3,"[50, 44, 32, 26, 39, 78, 19, 31, 40, 47]"
4,4,"[8, 39, 55, 87, 69, 101, 100, 22, 67, 21]"


In [15]:
df3 = one_row.hero_id.apply(pd.Series)
df3.columns = ['Radiant1', 'Radiant2','Radiant3','Radiant4','Radiant5','Dire1','Dire2','Dire3','Dire4','Dire5']
df3

,Radiant1,Radiant2,Radiant3,Radiant4,Radiant5,Dire1,Dire2,Dire3,Dire4,Dire5
0,86,51,83,11,67,106,102,46,7,73
1,7,82,71,39,21,73,22,5,67,106
2,51,109,9,41,27,38,7,10,12,85
3,50,44,32,26,39,78,19,31,40,47
4,8,39,55,87,69,101,100,22,67,21
...,...,...,...,...,...,...,...,...,...,...
49995,73,86,21,20,14,32,7,109,35,112
49996,93,74,100,32,85,36,1,112,60,71
49997,100,68,75,39,44,28,102,21,9,23
49998,56,50,2,72,30,46,7,29,44,3


In [16]:
data.drop_duplicates(subset='match_id', keep="first",inplace=True)
data=data.reset_index(drop=True)[["radiant_win"]]
data.head()

,radiant_win
0,True
1,False
2,False
3,False
4,True


In [17]:
data=pd.concat([data,df3],axis=1)
data.head()

,radiant_win,Radiant1,Radiant2,Radiant3,Radiant4,Radiant5,Dire1,Dire2,Dire3,Dire4,Dire5
0,True,86,51,83,11,67,106,102,46,7,73
1,False,7,82,71,39,21,73,22,5,67,106
2,False,51,109,9,41,27,38,7,10,12,85
3,False,50,44,32,26,39,78,19,31,40,47
4,True,8,39,55,87,69,101,100,22,67,21


In [18]:
data["radiant_win"]=data["radiant_win"].astype('int32')

In [19]:
data.head()

,radiant_win,Radiant1,Radiant2,Radiant3,Radiant4,Radiant5,Dire1,Dire2,Dire3,Dire4,Dire5
0,1,86,51,83,11,67,106,102,46,7,73
1,0,7,82,71,39,21,73,22,5,67,106
2,0,51,109,9,41,27,38,7,10,12,85
3,0,50,44,32,26,39,78,19,31,40,47
4,1,8,39,55,87,69,101,100,22,67,21


In [20]:
data_size=len(data["Radiant1"].unique())+10
data_size


121

In [21]:
sorted(data["Radiant1"].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 109,
 110,
 111,
 112]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    data[['Radiant1', 'Radiant2','Radiant3','Radiant4','Radiant5','Dire1','Dire2','Dire3','Dire4','Dire5']], data["radiant_win"], test_size=0.2)

In [23]:
type(X_train)

pandas.core.frame.DataFrame

In [24]:
data_numpy=torch.tensor(X_train[['Radiant1', 'Radiant2','Radiant3','Radiant4','Radiant5','Dire1','Dire2','Dire3','Dire4','Dire5']].to_numpy(),dtype=(torch.long))
print(data_numpy[:5])
labels=torch.tensor(y_train.to_numpy(),dtype=(torch.long))
print(labels[:5])
print(len(labels))

tensor([[ 71,  74,  30,  67,  53,  62,  65,  63,  60,  25],
        [  8,  73,  37, 100,  19,  86,  88,  21,  93,   7],
        [ 39,  69,   8,  68,  71, 110,  73,  86,  72,  50],
        [ 71,  15,  31,  76,  12, 106,   3,   9,  46,  69],
        [ 88,  14,  74, 104,  47,  18,  25,   9,  93,  21]])
tensor([1, 0, 1, 1, 0])
40000


In [25]:
modelemb= nn.Sequential(
    nn.Embedding(data_size, 15),
    nn.Flatten(),
    nn.BatchNorm1d(15*10),
    nn.Linear(15 * 10, 1),
    nn.Sigmoid()
)

In [26]:
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.AdamW(modelemb.parameters())

In [27]:
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = modelemb(data_numpy.long())
    loss = criterion(outputs.squeeze(), torch.tensor(labels, dtype=torch.float))
    loss.backward()
    optimizer.step()
    print(f"Epoch:{epoch}")
    print(f"Loss:{loss.item()}")
    print("___________________")

Epoch:0


<ipython-input-27-137e307aa58f>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = criterion(outputs.squeeze(), torch.tensor(labels, dtype=torch.float))


Loss:0.7314654588699341
___________________
Epoch:1
Loss:0.7295438051223755
___________________
Epoch:2
Loss:0.7276917099952698
___________________
Epoch:3
Loss:0.7259085774421692
___________________
Epoch:4
Loss:0.7241934537887573
___________________
Epoch:5
Loss:0.7225451469421387
___________________
Epoch:6
Loss:0.7209619879722595
___________________
Epoch:7
Loss:0.7194423079490662
___________________
Epoch:8
Loss:0.7179840803146362
___________________
Epoch:9
Loss:0.7165849804878235
___________________
Epoch:10
Loss:0.7152430415153503
___________________
Epoch:11
Loss:0.713956356048584
___________________
Epoch:12
Loss:0.7127225399017334
___________________
Epoch:13
Loss:0.7115395665168762
___________________
Epoch:14
Loss:0.7104053497314453
___________________
Epoch:15
Loss:0.7093178629875183
___________________
Epoch:16
Loss:0.7082751989364624
___________________
Epoch:17
Loss:0.707275390625
___________________
Epoch:18
Loss:0.7063168883323669
___________________
Epoch:19
Loss:0.

Loss:0.676401674747467
___________________
Epoch:156
Loss:0.6762886643409729
___________________
Epoch:157
Loss:0.6761756539344788
___________________
Epoch:158
Loss:0.6760625243186951
___________________
Epoch:159
Loss:0.6759495139122009
___________________
Epoch:160
Loss:0.6758363246917725
___________________
Epoch:161
Loss:0.6757232546806335
___________________
Epoch:162
Loss:0.6756101250648499
___________________
Epoch:163
Loss:0.6754969358444214
___________________
Epoch:164
Loss:0.6753838658332825
___________________
Epoch:165
Loss:0.675270676612854
___________________
Epoch:166
Loss:0.6751575469970703
___________________
Epoch:167
Loss:0.6750444173812866
___________________
Epoch:168
Loss:0.6749313473701477
___________________
Epoch:169
Loss:0.6748182773590088
___________________
Epoch:170
Loss:0.6747052073478699
___________________
Epoch:171
Loss:0.6745920777320862
___________________
Epoch:172
Loss:0.6744791269302368
___________________
Epoch:173
Loss:0.6743662357330322
______

Epoch:308
Loss:0.6615967750549316
___________________
Epoch:309
Loss:0.6615306735038757
___________________
Epoch:310
Loss:0.6614651083946228
___________________
Epoch:311
Loss:0.6613999009132385
___________________
Epoch:312
Loss:0.6613352298736572
___________________
Epoch:313
Loss:0.6612710356712341
___________________
Epoch:314
Loss:0.6612071990966797
___________________
Epoch:315
Loss:0.6611440181732178
___________________
Epoch:316
Loss:0.661081075668335
___________________
Epoch:317
Loss:0.6610186696052551
___________________
Epoch:318
Loss:0.6609567403793335
___________________
Epoch:319
Loss:0.6608952283859253
___________________
Epoch:320
Loss:0.6608340740203857
___________________
Epoch:321
Loss:0.6607734560966492
___________________
Epoch:322
Loss:0.660713255405426
___________________
Epoch:323
Loss:0.6606535315513611
___________________
Epoch:324
Loss:0.6605941653251648
___________________
Epoch:325
Loss:0.6605353355407715
___________________
Epoch:326
Loss:0.6604768633842

Loss:0.6558902263641357
___________________
Epoch:462
Loss:0.6558746099472046
___________________
Epoch:463
Loss:0.6558590531349182
___________________
Epoch:464
Loss:0.6558436751365662
___________________
Epoch:465
Loss:0.6558284163475037
___________________
Epoch:466
Loss:0.6558133363723755
___________________
Epoch:467
Loss:0.6557984948158264
___________________
Epoch:468
Loss:0.6557837128639221
___________________
Epoch:469
Loss:0.6557691693305969
___________________
Epoch:470
Loss:0.655754566192627
___________________
Epoch:471
Loss:0.6557403802871704
___________________
Epoch:472
Loss:0.6557262539863586
___________________
Epoch:473
Loss:0.6557121872901917
___________________
Epoch:474
Loss:0.6556983590126038
___________________
Epoch:475
Loss:0.6556846499443054
___________________
Epoch:476
Loss:0.6556710600852966
___________________
Epoch:477
Loss:0.6556577086448669
___________________
Epoch:478
Loss:0.6556443572044373
___________________
Epoch:479
Loss:0.6556311249732971
_____

Epoch:615
Loss:0.6544893383979797
___________________
Epoch:616
Loss:0.6544830799102783
___________________
Epoch:617
Loss:0.6544766426086426
___________________
Epoch:618
Loss:0.6544703245162964
___________________
Epoch:619
Loss:0.6544639468193054
___________________
Epoch:620
Loss:0.6544576287269592
___________________
Epoch:621
Loss:0.6544512510299683
___________________
Epoch:622
Loss:0.6544449329376221
___________________
Epoch:623
Loss:0.6544386148452759
___________________
Epoch:624
Loss:0.6544322967529297
___________________
Epoch:625
Loss:0.6544259786605835
___________________
Epoch:626
Loss:0.6544196009635925
___________________
Epoch:627
Loss:0.6544133424758911
___________________
Epoch:628
Loss:0.6544070243835449
___________________
Epoch:629
Loss:0.6544007658958435
___________________
Epoch:630
Loss:0.6543945074081421
___________________
Epoch:631
Loss:0.6543881893157959
___________________
Epoch:632
Loss:0.6543819308280945
___________________
Epoch:633
Loss:0.65437567234

Loss:0.6535452008247375
___________________
Epoch:769
Loss:0.6535390615463257
___________________
Epoch:770
Loss:0.6535329222679138
___________________
Epoch:771
Loss:0.6535268425941467
___________________
Epoch:772
Loss:0.6535207033157349
___________________
Epoch:773
Loss:0.653514564037323
___________________
Epoch:774
Loss:0.6535084247589111
___________________
Epoch:775
Loss:0.6535024046897888
___________________
Epoch:776
Loss:0.653496265411377
___________________
Epoch:777
Loss:0.6534901261329651
___________________
Epoch:778
Loss:0.6534841060638428
___________________
Epoch:779
Loss:0.6534779071807861
___________________
Epoch:780
Loss:0.6534717679023743
___________________
Epoch:781
Loss:0.6534656286239624
___________________
Epoch:782
Loss:0.6534594893455505
___________________
Epoch:783
Loss:0.6534534096717834
___________________
Epoch:784
Loss:0.6534473896026611
___________________
Epoch:785
Loss:0.6534411907196045
___________________
Epoch:786
Loss:0.6534351706504822
______

Epoch:921
Loss:0.6526190638542175
___________________
Epoch:922
Loss:0.6526131629943848
___________________
Epoch:923
Loss:0.6526072025299072
___________________
Epoch:924
Loss:0.6526013016700745
___________________
Epoch:925
Loss:0.6525953412055969
___________________
Epoch:926
Loss:0.6525894403457642
___________________
Epoch:927
Loss:0.6525834798812866
___________________
Epoch:928
Loss:0.6525775194168091
___________________
Epoch:929
Loss:0.6525715589523315
___________________
Epoch:930
Loss:0.6525657176971436
___________________
Epoch:931
Loss:0.652559757232666
___________________
Epoch:932
Loss:0.652553915977478
___________________
Epoch:933
Loss:0.6525478959083557
___________________
Epoch:934
Loss:0.6525421142578125
___________________
Epoch:935
Loss:0.652536153793335
___________________
Epoch:936
Loss:0.6525301933288574
___________________
Epoch:937
Loss:0.6525242924690247
___________________
Epoch:938
Loss:0.6525183320045471
___________________
Epoch:939
Loss:0.65251249074935

In [28]:
embeddings=list(modelemb[0].parameters())[0]
embeddings[:5]

tensor([[-2.2257,  0.5949,  0.4770,  2.0268, -1.2480, -1.3774, -3.1464,  2.1765,
          0.1908, -2.2394,  0.4046, -1.1960, -0.9957,  0.4171,  0.3456],
        [ 0.6076, -1.4441, -0.4170,  2.6588, -1.5536, -1.4166,  1.7525, -1.2560,
          1.0865, -1.3854,  1.3008, -0.7778,  1.5958,  0.0983,  0.5017],
        [-0.5293,  0.6957,  1.1857, -0.5401, -0.8959, -1.0740,  0.0363,  0.0137,
          0.3968,  0.2901, -1.4893, -0.7288, -1.0630,  0.4484, -0.7116],
        [-0.8806,  2.9346, -0.0638, -0.4809, -0.0718, -0.8453,  0.9828, -1.1196,
          0.1944,  0.4415, -0.0063,  1.2198, -0.7421, -0.2109, -1.7832],
        [-0.6759,  0.2464, -0.9186,  0.1559,  0.8134, -0.8796, -2.0011,  1.0789,
          1.1105, -0.9642,  0.0072,  1.3911,  0.3949, -0.8311, -1.0298]],
       grad_fn=<SliceBackward0>)

In [29]:
train_embeds=embeddings[data_numpy]
test_embeds=embeddings[torch.tensor(X_test[['Radiant1', 'Radiant2','Radiant3','Radiant4','Radiant5','Dire1','Dire2','Dire3','Dire4','Dire5']].to_numpy(),dtype=(torch.long))]

In [30]:
train_embeds.shape, test_embeds.shape

(torch.Size([40000, 10, 15]), torch.Size([10000, 10, 15]))

In [31]:
#flatten
train_embeds=np.array([i.flatten().detach().cpu().numpy() for i in train_embeds])
test_embeds=np.array([i.flatten().detach().cpu().numpy() for i in test_embeds])

In [32]:
type(train_embeds),train_embeds.shape

(numpy.ndarray, (40000, 150))

In [33]:
X_train_num=train_embeds
X_test_num=test_embeds

Logistic, XGboost, random forest, ann

## Logistic Regression

In [34]:
modellog=LogisticRegression()
modellog.fit(X_train_num,y_train)

LogisticRegression()

In [35]:
modellog.predict([X_train_num[0]])

array([1])

In [36]:
modellog.score(X_test_num,y_test)

0.5902

In [37]:
modellog.score(X_train_num,y_train)

0.617075

## random forest

In [38]:
X_randfor=X_train.to_numpy().reshape(X_train.shape[0],-1)
X_randfor.shape

(40000, 10)

In [39]:
modelforest=RandomForestClassifier(n_estimators=100)
modelforest.fit(X_randfor,y_train)

RandomForestClassifier()

In [40]:
modelforest.predict([X_randfor[0]]),modelforest.score(X_test,y_test)

(array([1]), 0.5151)

In [41]:
modelforest2=RandomForestClassifier(n_estimators=100)
modelforest2.fit(X_train_num,y_train)

RandomForestClassifier()

In [42]:
modelforest2.predict([X_train_num[0]]),modelforest2.score(X_test_num,y_test)

(array([1]), 0.5545)

In [43]:
modelforest2.score(X_train_num,y_train)

1.0

## XGBoost

In [44]:
xgb_train = xgb.DMatrix(X_train_num, y_train)
xgb_test = xgb.DMatrix(X_test_num, y_test)

In [45]:
n=200
params = {
    'objective': 'binary:logistic',
    'max_depth': 7,
    'learning_rate': 0.3,
}

In [46]:
modelxgb = xgb.train(params=params,dtrain=xgb_train,num_boost_round=n)

In [47]:
preds = modelxgb.predict(xgb_test)
preds = preds.astype(int)
accuracy= accuracy_score(y_test,preds)
accuracy

0.4859

In [48]:
xgb_train1 = xgb.DMatrix(X_train, y_train)
xgb_test1 = xgb.DMatrix(X_test, y_test)

In [49]:
n=200
params = {
    'objective': 'binary:logistic',
    'max_depth': 7,
    'learning_rate': 0.3,
}

In [50]:
modelxgb1 = xgb.train(params=params,dtrain=xgb_train1,num_boost_round=n)

In [51]:
preds = modelxgb1.predict(xgb_test1)
preds = preds.astype(int)
accuracy= accuracy_score(y_test,preds)
accuracy

0.4859

In [52]:
preds = modelxgb1.predict(xgb_train1)
preds = preds.astype(int)
accuracy= accuracy_score(y_train,preds)
accuracy

0.47995

## ANN

In [61]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Linear(in_features=150,out_features=16),
            nn.Dropout(0.5),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(in_features=16,out_features=4),
            nn.Dropout(0.5),
            nn.BatchNorm1d(4),
            nn.ReLU(),
            nn.Linear(in_features=4,out_features=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layer1(x)

modelNN=BinaryClassification()
modelNN

BinaryClassification(
  (layer1): Sequential(
    (0): Linear(in_features=150, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=4, bias=True)
    (5): Dropout(p=0.5, inplace=False)
    (6): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Linear(in_features=4, out_features=1, bias=True)
    (9): Sigmoid()
  )
)

In [62]:
loss = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.SGD(modelNN.parameters(),lr=0.05)

In [63]:
epochs = 500
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = modelNN(torch.tensor(X_train_num))
    losstrain = loss(outputs.squeeze(), torch.tensor(y_train, dtype=torch.float))
    losstrain.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"Epoch:{epoch}")
    print(f"Train Loss:{losstrain.item()}")
    
    modelNN.eval()
    
    with torch.inference_mode():
        outputs=modelNN(torch.tensor(X_test_num))
        losstest=loss(outputs.squeeze(),torch.tensor(y_test.to_numpy(),dtype=torch.float))
        acc = (outputs.round() == torch.tensor(y_test.to_numpy(),dtype=torch.float)).float().mean()
    print(f"Test Loss:{losstest.item()}")    
    print(f"Accuracy:{acc}")
    print("___________________")

Epoch:0
Train Loss:0.7202391624450684
Test Loss:0.7220606207847595
Accuracy:0.48590001463890076
___________________
Epoch:1
Train Loss:0.7239857316017151
Test Loss:0.7210315465927124
Accuracy:0.48590001463890076
___________________
Epoch:2
Train Loss:0.7228988409042358
Test Loss:0.7200323343276978
Accuracy:0.48590001463890076
___________________
Epoch:3
Train Loss:0.7218424081802368
Test Loss:0.7190610766410828
Accuracy:0.48590001463890076
___________________
Epoch:4
Train Loss:0.7208149433135986
Test Loss:0.7181178331375122
Accuracy:0.48590001463890076
___________________
Epoch:5
Train Loss:0.7198144793510437
Test Loss:0.7172001600265503
Accuracy:0.48590001463890076
___________________
Epoch:6
Train Loss:0.7188405990600586
Test Loss:0.7163101434707642
Accuracy:0.48590001463890076
___________________
Epoch:7
Train Loss:0.7178934812545776
Test Loss:0.7154463529586792
Accuracy:0.48590001463890076
___________________
Epoch:8
Train Loss:0.7169732451438904
Test Loss:0.7146065831184387
Accur

Test Loss:0.6940187811851501
Accuracy:0.49492964148521423
___________________
Epoch:72
Train Loss:0.6931919455528259
Test Loss:0.6939557790756226
Accuracy:0.49528780579566956
___________________
Epoch:73
Train Loss:0.6931092739105225
Test Loss:0.6938954591751099
Accuracy:0.4955754280090332
___________________
Epoch:74
Train Loss:0.6930297613143921
Test Loss:0.6938378214836121
Accuracy:0.49591100215911865
___________________
Epoch:75
Train Loss:0.6929534077644348
Test Loss:0.6937828660011292
Accuracy:0.4961845576763153
___________________
Epoch:76
Train Loss:0.6928800940513611
Test Loss:0.6937304139137268
Accuracy:0.4964863061904907
___________________
Epoch:77
Train Loss:0.6928094625473022
Test Loss:0.6936805248260498
Accuracy:0.4967767298221588
___________________
Epoch:78
Train Loss:0.6927415728569031
Test Loss:0.6936326026916504
Accuracy:0.49703335762023926
___________________
Epoch:79
Train Loss:0.6926762461662292
Test Loss:0.6935869455337524
Accuracy:0.4973520040512085
___________

Test Loss:0.6924618482589722
Accuracy:0.5065987706184387
___________________
Epoch:143
Train Loss:0.690798819065094
Test Loss:0.6924522519111633
Accuracy:0.5066410899162292
___________________
Epoch:144
Train Loss:0.6907821297645569
Test Loss:0.6924426555633545
Accuracy:0.5067031383514404
___________________
Epoch:145
Train Loss:0.6907654404640198
Test Loss:0.6924330592155457
Accuracy:0.5067651867866516
___________________
Epoch:146
Train Loss:0.6907488703727722
Test Loss:0.6924235224723816
Accuracy:0.5068046450614929
___________________
Epoch:147
Train Loss:0.6907323002815247
Test Loss:0.6924139857292175
Accuracy:0.5068526268005371
___________________
Epoch:148
Train Loss:0.6907159090042114
Test Loss:0.6924044489860535
Accuracy:0.5068836212158203
___________________
Epoch:149
Train Loss:0.690699577331543
Test Loss:0.6923949718475342
Accuracy:0.5069146156311035
___________________
Epoch:150
Train Loss:0.6906831860542297
Test Loss:0.6923854351043701
Accuracy:0.5069597363471985
_________

Test Loss:0.691749095916748
Accuracy:0.5080173015594482
___________________
Epoch:214
Train Loss:0.6896576285362244
Test Loss:0.6917383074760437
Accuracy:0.5080285668373108
___________________
Epoch:215
Train Loss:0.6896411180496216
Test Loss:0.6917275190353394
Accuracy:0.5080285668373108
___________________
Epoch:216
Train Loss:0.6896244287490845
Test Loss:0.6917164921760559
Accuracy:0.5080313682556152
___________________
Epoch:217
Train Loss:0.6896079182624817
Test Loss:0.691705584526062
Accuracy:0.5080285668373108
___________________
Epoch:218
Train Loss:0.6895912885665894
Test Loss:0.6916945576667786
Accuracy:0.5080285668373108
___________________
Epoch:219
Train Loss:0.6895745396614075
Test Loss:0.6916835904121399
Accuracy:0.5080341696739197
___________________
Epoch:220
Train Loss:0.6895577907562256
Test Loss:0.6916725039482117
Accuracy:0.5080398321151733
___________________
Epoch:221
Train Loss:0.6895410418510437
Test Loss:0.6916614770889282
Accuracy:0.5080482959747314
_________

Test Loss:0.6909062266349792
Accuracy:0.5077747106552124
___________________
Epoch:285
Train Loss:0.6883814334869385
Test Loss:0.6908932328224182
Accuracy:0.5077634453773499
___________________
Epoch:286
Train Loss:0.6883618235588074
Test Loss:0.690880298614502
Accuracy:0.5077521800994873
___________________
Epoch:287
Train Loss:0.6883421540260315
Test Loss:0.6908673048019409
Accuracy:0.5077493786811829
___________________
Epoch:288
Train Loss:0.6883223652839661
Test Loss:0.6908542513847351
Accuracy:0.5077380537986755
___________________
Epoch:289
Train Loss:0.6883025169372559
Test Loss:0.6908410787582397
Accuracy:0.5077324509620667
___________________
Epoch:290
Train Loss:0.6882827281951904
Test Loss:0.6908279061317444
Accuracy:0.5077239871025085
___________________
Epoch:291
Train Loss:0.6882628202438354
Test Loss:0.6908146739006042
Accuracy:0.5077183246612549
___________________
Epoch:292
Train Loss:0.6882427930831909
Test Loss:0.6908015012741089
Accuracy:0.5077098608016968
________

Test Loss:0.6898949146270752
Accuracy:0.5070274472236633
___________________
Epoch:356
Train Loss:0.686832845211029
Test Loss:0.6898791193962097
Accuracy:0.5070246458053589
___________________
Epoch:357
Train Loss:0.6868084669113159
Test Loss:0.6898632645606995
Accuracy:0.5070133209228516
___________________
Epoch:358
Train Loss:0.686784029006958
Test Loss:0.689847469329834
Accuracy:0.5069991946220398
___________________
Epoch:359
Train Loss:0.6867595911026001
Test Loss:0.6898316144943237
Accuracy:0.5069963932037354
___________________
Epoch:360
Train Loss:0.6867349147796631
Test Loss:0.6898157000541687
Accuracy:0.5069851279258728
___________________
Epoch:361
Train Loss:0.6867101788520813
Test Loss:0.6897997260093689
Accuracy:0.5069795250892639
___________________
Epoch:362
Train Loss:0.6866853833198547
Test Loss:0.6897835731506348
Accuracy:0.5069654583930969
___________________
Epoch:363
Train Loss:0.6866604685783386
Test Loss:0.6897674798965454
Accuracy:0.5069484710693359
__________

Test Loss:0.6886484026908875
Accuracy:0.5061588883399963
___________________
Epoch:427
Train Loss:0.6848933696746826
Test Loss:0.6886292099952698
Accuracy:0.5061390995979309
___________________
Epoch:428
Train Loss:0.6848629713058472
Test Loss:0.6886098384857178
Accuracy:0.5061250329017639
___________________
Epoch:429
Train Loss:0.6848323345184326
Test Loss:0.6885905861854553
Accuracy:0.5061165690422058
___________________
Epoch:430
Train Loss:0.6848017573356628
Test Loss:0.6885712146759033
Accuracy:0.5061137676239014
___________________
Epoch:431
Train Loss:0.6847710013389587
Test Loss:0.6885517835617065
Accuracy:0.5061053037643433
___________________
Epoch:432
Train Loss:0.6847401261329651
Test Loss:0.6885321736335754
Accuracy:0.5060997009277344
___________________
Epoch:433
Train Loss:0.6847092509269714
Test Loss:0.6885125041007996
Accuracy:0.506088376045227
___________________
Epoch:434
Train Loss:0.6846781969070435
Test Loss:0.6884926557540894
Accuracy:0.5060742497444153
________

Test Loss:0.68715900182724
Accuracy:0.5053043961524963
___________________
Epoch:498
Train Loss:0.6825018525123596
Test Loss:0.6871366500854492
Accuracy:0.5052847266197205
___________________
Epoch:499
Train Loss:0.6824652552604675
Test Loss:0.6871142387390137
Accuracy:0.5052734613418579
___________________


In [64]:
modelNN.eval()
    
with torch.inference_mode():
    outputs=modelNN(torch.tensor(X_test_num))
    losstest=loss(outputs.squeeze(),torch.tensor(y_test.to_numpy(),dtype=torch.float))
    acc = (outputs.round() == torch.tensor(y_test.to_numpy(),dtype=torch.float)).float().mean()
print(f"Test Loss:{losstest.item()}")    
print(f"Accuracy:{acc}")

with torch.inference_mode():
    outputs=modelNN(torch.tensor(X_train_num))
    losstest=loss(outputs.squeeze(),torch.tensor(y_train.to_numpy(),dtype=torch.float))
    acc = (outputs.round() == torch.tensor(y_train.to_numpy(),dtype=torch.float)).float().mean()
print(f"Train Loss:{losstest.item()}")    
print(f"Accuracy:{acc}")

Test Loss:0.6871142387390137
Accuracy:0.5052734613418579
Train Loss:0.6824284791946411
Accuracy:0.5078516006469727


Linear Regression performed the best ig